<a href="https://colab.research.google.com/github/kairok/BackendDjango/blob/master/IngateKoibagarov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #Библиотека работы с массивами
import pandas as pd # Библиотека для работы с базами

from tensorflow.keras.models import Sequential, Model #
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten #
from tensorflow.keras import utils #Используем для to_categoricall
from tensorflow.keras.optimizers import Adam,Adadelta,SGD,Adagrad,RMSprop #
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence #
from tensorflow.keras.preprocessing.sequence import pad_sequences #
from tensorflow.keras.callbacks import LambdaCallback # подключаем колбэки

from sklearn.preprocessing import StandardScaler #
from sklearn.model_selection import train_test_split # Для разбивки на выборки
from sklearn.metrics import mean_squared_error, mean_absolute_error #
#from google.colab import files #Для загрузки своей картинки

import random #Для генерации случайных чисел
import math # Для округления
#import os #Для работы с файлами
import re #
import matplotlib.pyplot as plt #
%matplotlib inline

from IPython.display import clear_output

In [ ]:
from google.colab import drive # Подключаем диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_name = "/content/drive/MyDrive/Базы/Аналитика Графики 01.06.2022-31.05.2023.xlsx"
df = pd.read_excel(file_name) #Загружаем данные в data frame
#data = df.values #Вытаскиваем данные в numpy array

In [ ]:
df.head(6) #Показываем пример данных (первые 6 колонок)

,Категория 3-го уровня,Товары,Модель,Ozon ID,Артикул,День,"Уникальные посетители, всего",Уникальные посетители с просмотром карточки товара,"Показы, всего",Показы на карточке товара,Показы в поиске и каталоге,Позиция в поиске и каталоге,"В корзину, всего",Заказано товаров,Отменено товаров,Доставлено (за выбранный период),Возвращено товаров,Заказано на сумму,"Отправления с премиум-подпиской, принятые в обработку"
0,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353426,34414,2023-03-03,1,1,6,1,4,"1,00",0,0,0,0,0,0,0
1,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353426,34414,2023-03-05,1,0,2,0,2,"1,00",0,0,0,0,0,0,0
2,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353427,34414,2023-03-05,65,3,69,3,40,"110,33",0,0,0,0,0,0,0
3,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353427,34414,2023-03-06,85,3,109,3,46,"103,43",0,0,0,0,0,0,0
4,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353427,34414,2023-03-07,44,2,50,2,37,"97,14",0,0,0,0,0,0,0
5,Разное автопринадлежности,Светодиодная подсветка салона,Светодиодная подсветка салона,882353427,34414,2023-03-08,47,1,54,2,47,"115,06",0,0,0,0,0,0,0


In [ ]:
print(df.columns) #Показываем имена колонок данных

Index(['Категория 3-го уровня', 'Товары', 'Модель', 'Ozon ID', 'Артикул',
       'День', 'Уникальные посетители, всего',
       'Уникальные посетители с просмотром карточки товара', 'Показы, всего',
       'Показы на карточке товара', 'Показы в поиске и каталоге',
       'Позиция в поиске и каталоге', 'В корзину, всего', 'Заказано товаров',
       'Отменено товаров', 'Доставлено (за выбранный период)',
       'Возвращено товаров', 'Заказано на сумму',
       'Отправления с премиум-подпиской, принятые в обработку'],
      dtype='object')


In [ ]:
print(df.corr())

<ipython-input-22-23236a4e6045>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.corr())


                                                     Ozon ID  \
Ozon ID                                             1.000000   
Уникальные посетители, всего                       -0.007220   
Уникальные посетители с просмотром карточки товара  0.004465   
Показы, всего                                      -0.010313   
Показы на карточке товара                          -0.000423   
Показы в поиске и каталоге                         -0.002272   
В корзину, всего                                   -0.000707   
Заказано товаров                                    0.013675   
Отменено товаров                                   -0.000895   
Доставлено (за выбранный период)                    0.014007   
Возвращено товаров                                  0.003326   
Заказано на сумму                                   0.001824   
Отправления с премиум-подпиской, принятые в обр... -0.001869   

                                                    Уникальные посетители, всего  \
Ozon ID            

Интересующий нас столбец Заказано товаров имеет наиболее сильную корреляцию, связь со полями Уникальные посетители с просмотром карточки товара          0.432788 и Показы на карточке товара  0.453615
